### Importing the Libraries

In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split

import gensim
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt 
import keras
from time import time

c:\users\kushagra\appdata\local\programs\python\python38\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


### Importing the Dataset

In [2]:
df = pd.read_csv("data/preprocessed_train.csv")
df

,textID,text,selected_text,sentiment
0,cb774db0d1,"['id', 'have', 'responded', 'if', 'i', 'were',...",id have responded if i were going,neutral
1,549e992a42,"['soon', 'sad', 'i', 'will', 'miss', 'you', 'h...",sooo sad,negative
2,088c60f138,"['my', 'boss', 'is', 'bullying', 'me']",bullying me,negative
3,9642c003ef,"['what', 'interview', 'leave', 'me', 'alone']",leave me alone,negative
4,358bd9e861,"['sons', 'of', 'why', 'couldnt', 'they', 'put'...",sons of,negative
...,...,...,...,...
27476,4eac33d1c0,"['wish', 'we', 'could', 'come', 'see', 'u', 'o...",d lost,negative
27477,4f4c4fc327,"['ive', 'wondered', 'about', 'rake', 'to', 'th...",dont force,negative
27478,f67aae2310,"['yay', 'good', 'for', 'both', 'of', 'you', 'e...",yay good for both of you,positive
27479,ed167662a5,"['but', 'it', 'was', 'worth', 'it']",but it was worth it,positive


In [3]:
#read data
df = pd.read_csv("data/preprocessed_train.csv")
df.text = df.selected_text.map(lambda x:str(x))
df.sentiment = df.sentiment.astype("category")
df.sentiment = df.sentiment.cat.codes

### Splitting into Training, Testing and Validation Sets

In [4]:
# train, val, test split
x_train, xtest, y_train, ytest = train_test_split(df.text.values, df.sentiment.values,stratify=df.sentiment.values, test_size=0.3,random_state=1)
y_train = to_categorical(y_train)
x_val = xtest[0:4122]
y_val = to_categorical(ytest[0:4122])
x_test = xtest[4122:]
y_test = ytest[4122:]


#tokenizing and padding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df.text.values)

X_train = tokenizer.texts_to_sequences(x_train)
X_val = tokenizer.texts_to_sequences(x_val)
X_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

word_index = tokenizer.word_index

In [5]:
#import glove embeddings

embeddings_index = {}
f = open(os.path.join('glove.twitter.27B.100d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

### Function to Train the CNN Model

In [6]:
#using glove

from keras.models import Sequential
from keras import regularizers
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv1D
from keras import layers
from sklearn.metrics import accuracy_score, f1_score

def cnn_glove(activation,optimizer,epochs,batchsize):
  embedding_dim = 100

  model = Sequential()
  model.add(layers.Embedding(input_dim=vocab_size, 
                          output_dim=embedding_dim,
                          weights=[embedding_matrix],
                          input_length=maxlen))
  model.add(Conv1D(32, kernel_size=2, activation=activation))
  model.add(layers.MaxPooling1D(2))
  model.add(Conv1D(32, kernel_size=2, activation=activation))
  model.add(Dropout(0.25))
  model.add(Conv1D(16, kernel_size=2, activation=activation))
  model.add(layers.MaxPooling1D(2))
  model.add(Conv1D(16, kernel_size=2, activation=activation))
  model.add(Dropout(0.25))

  model.add(Flatten())

  model.add(Dense(64, activation='tanh'))
  model.add(Dropout(0.5))

  model.add(Dense(3, activation='softmax'))
  model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  history = model.fit(X_train, y_train,
                    epochs=epochs,
                    verbose=0,
                    validation_data=(X_val, y_val),
                    batch_size=batchsize)
  
  return history, model

In [7]:
activation = ["selu", "elu", "sigmoid", "tanh"]
optimizer = ["adam", "SGD", "RMSprop", "Adadelta"]
epochs = [5,10,15,20]
batchsize = [8,16,32,64,128]

In [ ]:
# 1. selecting activation fixing - optimizer = adam, epochs = 5, batch = 16
sel_activation = {}
for i in activation:
  history, model = cnn_glove(i,"adam",5,16)
  temp = {i:model.evaluate(X_val,y_val)[1]}
  sel_activation.update(temp)
  keras.backend.clear_session()

sel_activation_final = max(sel_activation, key=sel_activation.get)
print("best activation function is ",sel_activation_final)

# 2. selecting optimizer by fixing - activation = best, epochs = 5, batch = 16
sel_optimizer = {}
for i in optimizer:
  history, model = cnn_glove(sel_activation_final,i,5,16)
  temp = {i:model.evaluate(X_val,y_val)[1]}
  sel_optimizer.update(temp)
  keras.backend.clear_session()

sel_optimizer_final = max(sel_optimizer, key=sel_optimizer.get)
print("best optimizer is ",sel_optimizer_final)

# 3.  graph epoch vs accuracy score

acc_train_epoch = {}
acc_val_epoch = {}
for i in epochs:
  history, model = cnn_glove(sel_activation_final,sel_optimizer_final,i,16)
  temp_train = {i:model.evaluate(X_train,y_train)[1]}
  temp_val = {i:model.evaluate(X_val,y_val)[1]}
  acc_train_epoch.update(temp_train)
  acc_val_epoch.update(temp_val)
  keras.backend.clear_session()

sel_epoch_final = max(acc_val_epoch, key=acc_val_epoch.get)
print("best epoch is ",sel_epoch_final)

df_epoch_train = pd.DataFrame(list(acc_train_epoch.items()), columns=['Epochs', 'Accuracy score'])
df_epoch_val = pd.DataFrame(list(acc_val_epoch.items()), columns=['Epochs', 'Accuracy score'])

df_epoch_val.Epochs = df_epoch_val.Epochs.map(lambda x:str(x))
df_epoch_train.Epochs = df_epoch_train.Epochs.map(lambda x:str(x))


plt.figure()
plt.plot(df_epoch_train.iloc[:,0],df_epoch_train.iloc[:,1],c="r",label="train",linestyle='--', marker='o')
plt.plot(df_epoch_val.iloc[:,0],df_epoch_val.iloc[:,1],c="b",label = "val",linestyle='--', marker='o')
plt.title("Accuracy score vs Epochs")
plt.ylabel("Accuracy score")
plt.xlabel("Epochs")
plt.legend()
#plt.savefig("images/acc-epoch-glove-"+case, bbox_inches='tight',dpi = 200)


# 4. graph batch size vs accuracy score
acc_train_batch = {}
acc_val_batch = {}
for i in batchsize:
  history, model = cnn_glove(sel_activation_final,sel_optimizer_final,sel_epoch_final,i)
  temp_train = {i:model.evaluate(X_train,y_train)[1]}
  temp_val = {i:model.evaluate(X_val,y_val)[1]}
  acc_train_batch.update(temp_train)
  acc_val_batch.update(temp_val)
  keras.backend.clear_session()

sel_batch_final = max(acc_val_batch, key=acc_val_batch.get)
print("best batchsize is ",sel_batch_final)

df_batch_train = pd.DataFrame(list(acc_train_batch.items()), columns=['Batchsize', 'Accuracy score'])
df_batch_val = pd.DataFrame(list(acc_val_batch.items()), columns=['Batchsize', 'Accuracy score'])

df_batch_val.Batchsize = df_batch_val.Batchsize.map(lambda x:str(x))
df_batch_train.Batchsize = df_batch_train.Batchsize.map(lambda x:str(x))

plt.figure()
plt.plot(df_batch_train.iloc[:,0],df_batch_train.iloc[:,1],c="r",label="train",linestyle='--', marker='o')
plt.plot(df_batch_val.iloc[:,0],df_batch_val.iloc[:,1],c="b",label = "val",linestyle='--', marker='o')
plt.title("Accuracy score vs Batchsize")
plt.ylabel("Accuracy score")
plt.xlabel("Batchsize")
plt.legend()
#plt.savefig("images/acc-batch-glove-"+case, bbox_inches='tight',dpi = 200)

In [8]:
#experiments using glove embeddings
sel_activation_final = 'selu'
print("best activation function is ",sel_activation_final)

sel_optimizer_final = 'RMSprop'
print("best optimizer is ",sel_optimizer_final)

sel_epoch_final = 5
print("best epoch is ",sel_epoch_final)

sel_batch_final = 8
print("best batchsize is ",sel_batch_final)

# Training the CNN Model
t0 = time()

history, model = cnn_glove(sel_activation_final,sel_optimizer_final,sel_epoch_final,sel_batch_final)
pred = np.argmax(model.predict(X_test), axis=-1)

print("test accuracy score = ",accuracy_score(y_pred=pred, y_true=y_test))
print("test f1 score = ",f1_score(y_pred=pred, y_true=y_test, average="weighted"))

t1 = time()
print("time taken is ", t1-t0)

best activation function is  selu
best optimizer is  RMSprop
best epoch is  5
best batchsize is  8
test accuracy score =  0.8430754305117633
test f1 score =  0.8418506662992482
time taken is  92.29925203323364


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1785200   
_________________________________________________________________
conv1d (Conv1D)              (None, 99, 32)            6432      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 48, 32)            2080      
_________________________________________________________________
dropout (Dropout)            (None, 48, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 47, 16)            1040      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 23, 16)            0

In [10]:
model.save("cnn_model.h5")

In [11]:
print(df.text[0])

id have responded if i were going


In [12]:
inp = ['Disappointing performance by CSK today',
'I`d have responded, if I were going',
'I hate the rain',
'I love the music',
'it was just the handle i guess'
]
X_test1 = tokenizer.texts_to_sequences(inp)
X_test1 = pad_sequences(X_test1, padding='post', maxlen = maxlen)
print('0: Negative\n1: Neutral\n2: Positive\n')
print(np.argmax(model.predict(X_test1), axis = -1))

0: Negative
1: Neutral
2: Positive

[0 1 0 2 1]
